In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
)

OUTDIR = Path("outputs_82HD")
OUTDIR.mkdir(parents=True, exist_ok=True)

lfw = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X = lfw.data
y = lfw.target
target_names = lfw.target_names
h, w = lfw.images.shape[1], lfw.images.shape[2]

n_samples, n_features = X.shape
n_classes = target_names.shape[0]
print(f"Dataset: LFW (min_faces_per_person=70, resize=0.4)")
print(
    f"Samples: {n_samples}, Features: {n_features} ({h}x{w} pixels), Classes: {n_classes}"
)
print("Classes:", ", ".join(target_names))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)


def evaluate_and_time(pipeline: Pipeline, name: str, report_prefix: str):
    t0 = time.perf_counter()
    pipeline.fit(X_train, y_train)
    t_train = time.perf_counter() - t0

    t1 = time.perf_counter()
    y_pred = pipeline.predict(X_test)
    t_test = time.perf_counter() - t1

    acc = accuracy_score(y_test, y_pred)
    pr, rc, f1, _ = precision_recall_fscore_support(
        y_test, y_pred, average="weighted", zero_division=0
    )
    print(f"\n{name}")
    print(f"Accuracy={acc:.4f}  Precision={pr:.4f}  Recall={rc:.4f}  F1={f1:.4f}")

    report_txt = classification_report(
        y_test, y_pred, target_names=target_names, zero_division=0
    )
    (OUTDIR / f"{report_prefix}_classification_report.txt").write_text(report_txt)
    cm = confusion_matrix(y_test, y_pred)
    return {
        "name": name,
        "acc": acc,
        "prec": pr,
        "rec": rc,
        "f1": f1,
        "t_train": t_train,
        "t_test": t_test,
        "cm": cm,
        "y_pred": y_pred,
    }


def save_cm(cm, title, filename):
    plt.figure(figsize=(6.5, 5.5))
    plt.imshow(cm, interpolation="nearest")
    plt.title(title)
    plt.colorbar()
    plt.tight_layout()
    plt.savefig(OUTDIR / filename, dpi=180)
    plt.close()


def plot_metric_vs_components(df, method_filter, metric, title, filename):
    subset = df[df["method"] == method_filter].copy()
    plt.figure(figsize=(7.0, 5.0))
    if method_filter == "PCA+SVM":
        xs = subset["pca"].values
        ys = subset[metric].values
        plt.plot(xs, ys, marker="o")
        plt.xlabel("PCA components")
    else:
        labels = sorted(subset["lda"].unique())
        for l in labels:
            sub2 = subset[subset["lda"] == l].sort_values("pca")
            plt.plot(
                sub2["pca"].values, sub2[metric].values, marker="o", label=f"LDA={l}"
            )
        plt.xlabel("PCA components (pre-LDA)")
        plt.legend()
    plt.ylabel(metric.upper())
    plt.title(title)
    plt.tight_layout()
    plt.savefig(OUTDIR / filename, dpi=180)
    plt.close()


def baseline_svm():
    return Pipeline(
        [
            ("scaler", StandardScaler(with_mean=True)),
            ("svm", SVC(kernel="rbf", class_weight="balanced", random_state=42)),
        ]
    )


def pca_svm(n_components: int):
    return Pipeline(
        [
            ("scaler", StandardScaler(with_mean=True)),
            (
                "pca",
                PCA(
                    n_components=n_components,
                    svd_solver="randomized",
                    whiten=True,
                    random_state=42,
                ),
            ),
            ("svm", SVC(kernel="rbf", class_weight="balanced", random_state=42)),
        ]
    )


def pca_lda_svm(n_pca: int, n_lda: int):
    n_lda = min(n_lda, n_classes - 1)
    return Pipeline(
        [
            ("scaler", StandardScaler(with_mean=True)),
            (
                "pca",
                PCA(
                    n_components=n_pca,
                    svd_solver="randomized",
                    whiten=True,
                    random_state=42,
                ),
            ),
            ("lda", LDA(n_components=n_lda, solver="svd")),
            ("svm", SVC(kernel="rbf", class_weight="balanced", random_state=42)),
        ]
    )


res_baseline = evaluate_and_time(
    baseline_svm(), "Baseline: SVM (no reduction)", "baseline"
)

pca_fixed_k = 150
res_pca_fixed = evaluate_and_time(
    pca_svm(pca_fixed_k), f"PCA({pca_fixed_k}) + SVM", "pca150"
)

lda_fixed_pca = 150
lda_fixed_lda = min(6, n_classes - 1)
res_lda_fixed = evaluate_and_time(
    pca_lda_svm(lda_fixed_pca, lda_fixed_lda),
    f"PCA({lda_fixed_pca}) -> LDA({lda_fixed_lda}) + SVM",
    "pca150_lda",
)

save_cm(res_baseline["cm"], "Confusion Matrix: Baseline SVM", "cm_baseline.png")
save_cm(
    res_pca_fixed["cm"], f"Confusion Matrix: PCA({pca_fixed_k})+SVM", "cm_pca150.png"
)
save_cm(
    res_lda_fixed["cm"],
    f"Confusion Matrix: PCA({lda_fixed_pca})->LDA({lda_fixed_lda})+SVM",
    "cm_lda.png",
)

pca150 = pca_svm(pca_fixed_k)
param_grid = {"svm__C": [1, 5, 10], "svm__gamma": ["scale", 1e-3, 1e-4]}
gs = GridSearchCV(pca150, param_grid, cv=3, scoring="f1_weighted", n_jobs=-1)
_ = gs.fit(X_train, y_train)
best_params_pca = gs.best_params_
print("\nGridSearch PCA(150)+SVM best params:", best_params_pca)

tuned_pca150 = pca_svm(pca_fixed_k)
tuned_pca150.set_params(
    **{"svm__C": best_params_pca["svm__C"], "svm__gamma": best_params_pca["svm__gamma"]}
)
res_pca_tuned = evaluate_and_time(
    tuned_pca150, f"PCA({pca_fixed_k}) + SVM [tuned]", "pca150_tuned"
)

lda_grid = {
    "svm__C": [1, 5],
    "svm__gamma": ["scale", 1e-3],
}
lda_pipe_for_gs = pca_lda_svm(lda_fixed_pca, lda_fixed_lda)
gs2 = GridSearchCV(lda_pipe_for_gs, lda_grid, cv=3, scoring="f1_weighted", n_jobs=-1)
_ = gs2.fit(X_train, y_train)
best_params_lda = gs2.best_params_
print("GridSearch PCA->LDA best params:", best_params_lda)

tuned_lda = pca_lda_svm(lda_fixed_pca, lda_fixed_lda)
tuned_lda.set_params(
    **{"svm__C": best_params_lda["svm__C"], "svm__gamma": best_params_lda["svm__gamma"]}
)
res_lda_tuned = evaluate_and_time(
    tuned_lda,
    f"PCA({lda_fixed_pca}) -> LDA({lda_fixed_lda}) + SVM [tuned]",
    "pca150_lda_tuned",
)

pca_components = [50, 100, 150, 200]
lda_lda_components = [2, 4, min(6, n_classes - 1)]
lda_pca_components = [80, 120, 150]

sweep_rows = []

for k in pca_components:
    res = evaluate_and_time(pca_svm(k), f"PCA({k}) + SVM", f"sweep_pca_{k}")
    sweep_rows.append(
        {
            "method": "PCA+SVM",
            "pca": k,
            "lda": 0,
            "acc": res["acc"],
            "prec": res["prec"],
            "rec": res["rec"],
            "f1": res["f1"],
            "t_train": res["t_train"],
            "t_test": res["t_test"],
        }
    )

for pca_k in lda_pca_components:
    for lda_k in lda_lda_components:
        res = evaluate_and_time(
            pca_lda_svm(pca_k, lda_k),
            f"PCA({pca_k}) -> LDA({lda_k}) + SVM",
            f"sweep_lda_{pca_k}_{lda_k}",
        )
        sweep_rows.append(
            {
                "method": "PCA->LDA+SVM",
                "pca": pca_k,
                "lda": lda_k,
                "acc": res["acc"],
                "prec": res["prec"],
                "rec": res["rec"],
                "f1": res["f1"],
                "t_train": res["t_train"],
                "t_test": res["t_test"],
            }
        )

df = pd.DataFrame(sweep_rows).sort_values(
    ["method", "f1", "acc"], ascending=[True, False, False]
)
print("\nSweep summary (sorted by method then F1):")
print(df.to_string(index=False))

df.to_csv(OUTDIR / "sweep_results.csv", index=False)


plot_metric_vs_components(
    df, "PCA+SVM", "f1", "PCA components vs F1 (PCA+SVM)", "pca_f1.png"
)

plot_metric_vs_components(
    df,
    "PCA->LDA+SVM",
    "f1",
    "PCA pre-dim vs F1 for different LDA dims (PCA->LDA+SVM)",
    "lda_f1.png",
)

plot_metric_vs_components(
    df, "PCA+SVM", "acc", "PCA components vs Accuracy (PCA+SVM)", "pca_acc.png"
)

plot_metric_vs_components(
    df,
    "PCA->LDA+SVM",
    "acc",
    "PCA pre-dim vs Accuracy for different LDA dims (PCA->LDA+SVM)",
    "lda_acc.png",
)


print("\nReproducibility:")
print("- Random seeds fixed with random_state=42 where applicable.")
print(f"- LDA max components enforced: n_classes-1 = {n_classes-1}.")
print("- PCA uses randomized SVD with whitening to stabilize downstream SVM.")
print(f"- Outputs saved under: {OUTDIR.resolve()}")

Dataset: LFW (min_faces_per_person=70, resize=0.4)
Samples: 1288, Features: 1850 (50x37 pixels), Classes: 7
Classes: Ariel Sharon, Colin Powell, Donald Rumsfeld, George W Bush, Gerhard Schroeder, Hugo Chavez, Tony Blair

Baseline: SVM (no reduction)
Accuracy=0.7795  Precision=0.7780  Recall=0.7795  F1=0.7747

PCA(150) + SVM
Accuracy=0.8292  Precision=0.8471  Recall=0.8292  F1=0.8216

PCA(150) -> LDA(6) + SVM
Accuracy=0.8447  Precision=0.8460  Recall=0.8447  F1=0.8447

GridSearch PCA(150)+SVM best params: {'svm__C': 5, 'svm__gamma': 0.001}

PCA(150) + SVM [tuned]
Accuracy=0.8509  Precision=0.8551  Recall=0.8509  F1=0.8499
GridSearch PCA->LDA best params: {'svm__C': 1, 'svm__gamma': 0.001}

PCA(150) -> LDA(6) + SVM [tuned]
Accuracy=0.8571  Precision=0.8586  Recall=0.8571  F1=0.8572

PCA(50) + SVM
Accuracy=0.8944  Precision=0.8985  Recall=0.8944  F1=0.8922

PCA(100) + SVM
Accuracy=0.8540  Precision=0.8680  Recall=0.8540  F1=0.8487

PCA(150) + SVM
Accuracy=0.8292  Precision=0.8471  Recall=